In [66]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from scipy.optimize import least_squares
import timeit
from scipy.spatial import distance
import pyransac3d as pyrsc

In [53]:
# Load the pickle files from data folder
points1 = np.load(f'data/3d_points_0.pkl', allow_pickle=True)
points2 = np.load(f'data/3d_points_1.pkl', allow_pickle=True)
points3 = np.load(f'data/3d_points_2.pkl', allow_pickle=True)

# Convert the points to open3d point cloud
pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(points1)
pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(points2)
pcd3 = o3d.geometry.PointCloud()
pcd3.points = o3d.utility.Vector3dVector(points3)

# Visualize the point clouds
o3d.visualization.draw_geometries([pcd1])
o3d.visualization.draw_geometries([pcd2])
o3d.visualization.draw_geometries([pcd3])

In [54]:
def remove_top_plane(points, percent=80, draw=False):
    # Find the upper height which is the height of the upper percentile of points in z direction
    upper_height = np.percentile(points[:,2], percent)
    
    inlier_points = points[np.where(points[:,2] < upper_height)]
    outlier_points = points[np.where(points[:,2] >= upper_height)]

    if draw:
        inlier_pcd = o3d.geometry.PointCloud()
        inlier_pcd.points = o3d.utility.Vector3dVector(inlier_points)
        outlier_pcd = o3d.geometry.PointCloud()
        outlier_pcd.points = o3d.utility.Vector3dVector(outlier_points)

        inlier_pcd.paint_uniform_color([1, 0, 0])
        outlier_pcd.paint_uniform_color([0, 1, 0])
        o3d.visualization.draw_geometries([inlier_pcd, outlier_pcd])

    return inlier_points

side_points1 = remove_top_plane(points1, draw=False)
side_points2 = remove_top_plane(points2, draw=False)
side_points3 = remove_top_plane(points3, draw=False)

In [78]:
def find_corner_points(points, thresh=0.1, min_pts_2planes=50, min_points=100, max_iter=1000, plot=False, plot_planes=False):
    # Calculate the mean z value of all points
    mean_z = np.mean(points[:,2])
    
    # Fit a plane to the points
    plane1 = pyrsc.Plane()
    best_eq1, best_inliers1 = plane1.fit(points, thresh=thresh, minPoints=min_points, maxIteration=max_iter)
    
    
    if plot_planes:
        plot_plane_fit(points, best_eq1, best_inliers1)
    
    # If there are enough points left, fit a second plane to the outliers or take the outermost points in first plane
    outliers = np.delete(points, best_inliers1, axis=0)
    print(f"Number of outliers for second plane: {len(outliers)}")
    if len(outliers) < min_pts_2planes:
        print("Not enough points for a second plane.")
        # Find the farthest two outermost points of the inliers on the plain
        dists = distance.cdist(points[best_inliers1], points[best_inliers1], 'euclidean') # pairwise distances
        
        # Get the indices of the two farthest points
        i, j = np.unravel_index(np.argmax(dists), dists.shape)
        point1, point2 = points[best_inliers1][i], points[best_inliers1][j]
        
        corner_points = np.vstack((point1, point2))
        
        corner_points = project_points_onto_plane(corner_points, best_eq1, mean_z)
        
        if plot:
            plot_segmented_points(points, best_inliers1, [], corner_points)
        
        return corner_points
        
    else:
        # Fit a second plane to the outliers and get 3 corner points
        plane2 = pyrsc.Plane()
        best_eq2, best_inliers2 = plane2.fit(outliers, thresh=thresh, minPoints=min_points)
        
        if plot_planes:
            plot_plane_fit(outliers, best_eq2, best_inliers2)
        
        # Find the intersection line of the two planes
        intersection_point, direction = find_intersection_line(best_eq1, best_eq2)
        
        # Find the point on the line at the mean z value of all points
        t = (mean_z - intersection_point[2]) / direction[2]
        corner_point = intersection_point + t * direction
        
        # Find the farthest point from the corner point projected to the plane for each plane
        z_value = corner_point[2]
        edge_point1 = get_edge_corner_point(points[best_inliers1], best_eq1, corner_point, z_value)
        edge_point2 = get_edge_corner_point(outliers[best_inliers2], best_eq2, corner_point, z_value)
        
        corner_points = np.vstack((corner_point, edge_point1, edge_point2))
        if plot:
            plot_segmented_points(points, best_inliers1, best_inliers2, corner_points)
            
        return corner_points

def project_points_onto_plane(points, plane_params, z_value):
    A, B, C, D = plane_params
    normal = np.array([A, B, C])
    normal_squared = np.dot(normal, normal)
    
    # Calculate the distance of each point from the plane
    distances = (np.dot(points, normal) + D) / normal_squared
    
    # Project each point onto the plane
    projections = points - np.outer(distances, normal)
    
    # Place the z value of the projected points at the z_value
    projections[:,2] = z_value
    
    return projections

def get_edge_corner_point(points, eq, corner_point, z_value):
    # Find the point on the plane that is farthest from the corner point
    A, B, C, D = eq
    normal = np.array([A, B, C])
    point = corner_point - D * normal / np.dot(normal, normal)
    distances = np.linalg.norm(points - point, axis=1)
    edge_point = points[np.argmax(distances)]
    
    # Place the z value of the edge point at the mean z value of all points
    edge_point[2] = z_value
    return edge_point

def find_intersection_line(plane1_params, plane2_params):
    A1, B1, C1, D1 = plane1_params
    A2, B2, C2, D2 = plane2_params

    # Normal vectors and constants
    A = np.array([[A1, B1, C1],
                [A2, B2, C2]])
    b = np.array([-D1, -D2])

    # Use least-squares to find the point that minimizes the distance to both planes
    point_on_line, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)

    # Direction vector for the line of intersection
    n1 = np.array([A1, B1, C1])
    n2 = np.array([A2, B2, C2])
    direction = np.cross(n1, n2)

    # Check if the direction is near zero (planes are parallel)
    if np.allclose(direction, 0):
        print("The planes are parallel and do not intersect.")
        return None, None

    return point_on_line, direction

def plot_segmented_points(points, inliers1, inliers2, corner_points):
    # Create point cloud of inliers for first plane fit
    inlier_pcd1 = o3d.geometry.PointCloud()
    inlier_pcd1.points = o3d.utility.Vector3dVector(points[inliers1])
    inlier_pcd1.paint_uniform_color([0, 1, 0])
    
    # Create point cloud of inliers for second plane fit (outliers of first plane fit)
    outliers = np.delete(points, inliers1, axis=0)
    inlier_pcd2 = o3d.geometry.PointCloud()
    inlier_pcd2.points = o3d.utility.Vector3dVector(outliers[inliers2])
    inlier_pcd2.paint_uniform_color([0, 1, 1])
    
    # Create point cloud of outliers for second plane fit (outliers of second plane fit)
    outlier_pcd = o3d.geometry.PointCloud()
    outlier_points = np.delete(outliers, inliers2, axis=0)
    outlier_pcd.points = o3d.utility.Vector3dVector(outlier_points)
    outlier_pcd.paint_uniform_color([1, 0, 0])
    
    # Create point cloud for final corner points
    corner_pcd = o3d.geometry.PointCloud()
    corner_pcd.points = o3d.utility.Vector3dVector(corner_points)
    corner_pcd.paint_uniform_color([1, 0, 1])
    
    o3d.visualization.draw_geometries([inlier_pcd1, inlier_pcd2, outlier_pcd, corner_pcd])

def plot_plane_fit(points, eq, inliers):
    inlier_pcd = o3d.geometry.PointCloud()
    inlier_pcd.points = o3d.utility.Vector3dVector(points[inliers])
    inlier_pcd.paint_uniform_color([0, 1, 0])
    
    outlier_pcd = o3d.geometry.PointCloud()
    outlier_points = np.delete(points, inliers, axis=0)
    outlier_pcd.points = o3d.utility.Vector3dVector(outlier_points)
    outlier_pcd.paint_uniform_color([1, 0, 0])
    
    plane_mesh = create_plane_mesh(eq)
    
    o3d.visualization.draw_geometries([inlier_pcd, outlier_pcd, plane_mesh])
    
def create_plane_mesh(plane_params, size=1, color=[0, 1, 0]):
    A, B, C, D = plane_params
    # Normal of the plane
    normal = np.array([A, B, C])
    # Create three points on the plane to define it
    u = np.cross(normal, [1, 0, 0])
    if np.linalg.norm(u) == 0:
        u = np.cross(normal, [0, 1, 0])
    u /= np.linalg.norm(u)
    v = np.cross(normal, u)
    u, v = u * size, v * size
    # Create four corners of the plane
    origin = -D * normal / np.dot(normal, normal)
    corners = [
        origin + u + v,
        origin + u - v,
        origin - u - v,
        origin - u + v
    ]
    # Create a mesh for the plane
    plane = o3d.geometry.TriangleMesh()
    plane.vertices = o3d.utility.Vector3dVector(corners)
    plane.triangles = o3d.utility.Vector3iVector([[0, 1, 2], [2, 3, 0]])
    plane.paint_uniform_color(color)
    return plane

In [79]:
start_time = timeit.default_timer()
find_corner_points(side_points2, thresh=0.1, min_points=100, plot=True, plot_planes=True)
print(timeit.default_timer() - start_time)

Number of outliers for second plane: 21
Not enough points for a second plane.
[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.
9.353098823999972


In [69]:
start_time = timeit.default_timer()
plane1 = pyrsc.Plane()
best_eq1, best_inliers = plane1.fit(points1_numpy_filtered, thresh=0.1, minPoints=3, maxIteration=1000)
print('Time: ', timeit.default_timer() - start_time)

# Plot the inliers in green and outliers in red
inliers = points1_numpy_filtered[best_inliers]
outliers = np.delete(points1_numpy_filtered, best_inliers, axis=0)
inlier_pcd = o3d.geometry.PointCloud()
inlier_pcd.points = o3d.utility.Vector3dVector(inliers)
inlier_pcd.paint_uniform_color([0, 1, 0])
outlier_pcd = o3d.geometry.PointCloud()
outlier_pcd.points = o3d.utility.Vector3dVector(outliers)
outlier_pcd.paint_uniform_color([1, 0, 0])
o3d.visualization.draw_geometries([inlier_pcd, outlier_pcd])

Time:  0.04799309500049276


In [70]:
plane2 = pyrsc.Plane()
best_eq2, best_inliers = plane2.fit(outliers, thresh=0.1, minPoints=3, maxIteration=1000)

# Plot the inliers in green and outliers in red
inliers = outliers[best_inliers]
outliers = np.delete(outliers, best_inliers, axis=0)
inlier_pcd = o3d.geometry.PointCloud()
inlier_pcd.points = o3d.utility.Vector3dVector(inliers)
inlier_pcd.paint_uniform_color([0, 1, 0])
outlier_pcd = o3d.geometry.PointCloud()
outlier_pcd.points = o3d.utility.Vector3dVector(outliers)
outlier_pcd.paint_uniform_color([1, 0, 0])
o3d.visualization.draw_geometries([inlier_pcd, outlier_pcd])

In [71]:
# Plot all points and the two planes
all_points_pcd = o3d.geometry.PointCloud()
all_points_pcd.points = o3d.utility.Vector3dVector(points1_numpy)
plane1_mesh = create_plane_mesh(best_eq1)
plane2_mesh = create_plane_mesh(best_eq2)

o3d.visualization.draw_geometries([all_points_pcd, plane1_mesh, plane2_mesh])

In [79]:


def create_line_segment(point, direction, length=10, color=[1, 0, 0]):
    line_points = [
        point - direction * length / 2,
        point + direction * length / 2
    ]
    line = o3d.geometry.LineSet()
    line.points = o3d.utility.Vector3dVector(line_points)
    line.lines = o3d.utility.Vector2iVector([[0, 1]])
    line.colors = o3d.utility.Vector3dVector([color])
    return line




In [81]:
# Plot all points and the two planes
all_points_pcd = o3d.geometry.PointCloud()
all_points_pcd.points = o3d.utility.Vector3dVector(points1_numpy)
point_on_line, direction = find_intersection_line(best_eq1, best_eq2)
line_segment = create_line_segment(point_on_line, direction, length=10)

# Visualize the mean height point on line
mean_intersection_point = mean_point_on_line(point_on_line, direction, points1_numpy[:,2].mean())

point_on_line_pcd = o3d.geometry.PointCloud()
point_on_line_pcd.points = o3d.utility.Vector3dVector([mean_intersection_point])
point_on_line_pcd.paint_uniform_color([1, 0, 0])

o3d.visualization.draw_geometries([all_points_pcd, line_segment, point_on_line_pcd])

In [40]:
# Residual function for a rectangular prism
def residuals(params, points):
    x0, y0, z0, L, W, H, theta = params
    # Rotation matrix for Z-axis rotation
    R = np.array([
        [np.cos(theta), -np.sin(theta), 0],
        [np.sin(theta), np.cos(theta), 0],
        [0, 0, 1]
    ])

    # Transform points to the prism's coordinate frame
    points_transformed = (points - np.array([x0, y0, z0])) @ R.T

    # Calculate distances to the prism's faces
    dx = np.abs(points_transformed[:, 0]) - L / 2
    dy = np.abs(points_transformed[:, 1]) - W / 2
    dz = np.abs(points_transformed[:, 2]) - H / 2

    # min_dist = np.min(np.stack([dx, dy, dz], axis=1), axis=1)
    # return min_dist**2


    # Only keep distances outside the box (inside distances are zero)
    dist_outside = np.maximum(np.maximum(dx, dy), dz)
    # return np.where(dist_outside > 0, dist_outside, 0)
    return np.abs(dist_outside)

# Initial guess for parameters
def initial_guess_rect_prism(points):
    x0, y0, z0 = np.mean(points, axis=0)
    L0, W0, H0 = np.ptp(points, axis=0)
    return [x0, y0, z0, L0, W0, H0, 0]

# Function to fit a rectangular prism using least squares
def fit_rect_prism(points):
    initial_guess = initial_guess_rect_prism(points)
    result = least_squares(residuals, initial_guess, args=(points,))
    return result.x

# RANSAC implementation for rectangular prism fitting
def ransac_rect_prism(points, threshold, iterations):
    best_inliers = []
    best_params = None

    for _ in range(iterations):
        # Randomly sample points to fit
        sample = points[np.random.choice(points.shape[0], 8, replace=False)]
        params = fit_rect_prism(sample)
        inliers = []

        # Compute inliers
        for point in points:
            distance = np.abs(residuals(params, point[np.newaxis]))
            if distance < threshold:
                inliers.append(point)

        # Update best model if the current one has more inliers
        if len(inliers) > len(best_inliers):
            best_inliers = inliers
            best_params = params

    return best_params, np.array(best_inliers)

In [22]:
# Function to create a rectangular prism given center, dimensions, and rotation angle
def create_rect_prism(x0, y0, z0, L, W, H, theta):
    # Rotation matrix for Z-axis rotation
    R = np.array([
        [np.cos(theta), -np.sin(theta), 0],
        [np.sin(theta), np.cos(theta), 0],
        [0, 0, 1]
    ])
    
    # Define the prism's corners relative to the center
    corners = np.array([
        [-L / 2, -W / 2, -H / 2],
        [L / 2, -W / 2, -H / 2],
        [L / 2, W / 2, -H / 2],
        [-L / 2, W / 2, -H / 2],
        [-L / 2, -W / 2, H / 2],
        [L / 2, -W / 2, H / 2],
        [L / 2, W / 2, H / 2],
        [-L / 2, W / 2, H / 2]
    ])
    
    # Rotate and translate the corners
    corners_transformed = (corners @ R.T) + np.array([x0, y0, z0])

    # Define the lines that connect the corners
    lines = [
        [0, 1], [1, 2], [2, 3], [3, 0],  # Bottom face
        [4, 5], [5, 6], [6, 7], [7, 4],  # Top face
        [0, 4], [1, 5], [2, 6], [3, 7]   # Vertical edges
    ]

    # Create a LineSet for visualization
    line_set = o3d.geometry.LineSet()
    line_set.points = o3d.utility.Vector3dVector(corners_transformed)
    line_set.lines = o3d.utility.Vector2iVector(lines)
    line_set.colors = o3d.utility.Vector3dVector([[1, 0, 0] for _ in lines])  # Red lines

    return line_set

In [42]:
start_time = timeit.default_timer()
best_params, inliers = ransac_rect_prism(points1_numpy, threshold=0.01, iterations=1000)
print("Time taken:", timeit.default_timer() - start_time)
print("Rectangular prism parameters:", best_params)

# Create Open3D PointCloud object for points
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(points1_numpy)
point_cloud.paint_uniform_color([0, 1, 0])  # Green points

# Generate rectangular prism LineSet
x0, y0, z0, L, W, H, theta = best_params
# theta = np.radians(-5)
rect_prism = create_rect_prism(x0, y0, z0, L, W, H, theta)

# Visualize
o3d.visualization.draw_geometries([point_cloud, rect_prism])

Time taken: 10.0837651490001
Rectangular prism parameters: [ 4.4487171   6.26904991 -1.51670056  4.32967107  3.83214521  1.41547579
  0.08562404]
